<a href="https://colab.research.google.com/github/Untick/resume-builder-gr2/blob/main/Malofeev%20Alexander/%D0%9C%D0%B0%D0%BB%D0%BE%D1%84%D0%B5%D0%B5%D0%B2_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%9F%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D0%B0_%D0%BE%D1%80%D1%84%D0%BE%D0%B3%D1%80%D0%B0%D1%84%D0%B8%D0%B8%2009%2009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Установка библиотек
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 chromadb gspread oauth2client nltk pydantic==1.10.8 faiss-cpu==1.7.4
!pip install diff-match-patch

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

import json
import diff_match_patch as dmp_module
from IPython.display import HTML

In [ ]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines11(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, temp = 0.1, verbose = 0):
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
    ]

    # example token count from the function defined above
    if (verbose): print('\n===========================================')
    if (verbose): print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0613')} токенов ориентировочная длина вопроса")
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp
    )
    price = (0.0015*completion["usage"]["prompt_tokens"]+0.002*completion["usage"]["completion_tokens"])/1000

    #if (verbose): print(f'ответ:\n{json.dumps(completion, indent=4)}')
    if (verbose): print('\n===========================================')
    if (verbose): print(f'{completion["usage"]["prompt_tokens"]}+{completion["usage"]["completion_tokens"]}={completion["usage"]["total_tokens"]} токенов использовано всего (вопрос+ответ).')
    if (verbose): print('\n===========================================')
    if (verbose): print(f'Цена запроса с ответом : $ {price:.4f}, {price*131.5:.2f} ₽')
    if (verbose): print('\n===========================================')

    response = completion.choices[0].message.content
    print(f'Исходное резюме:\n{response}')
    print(f'\n\n\nИсправленное резюме:\n{response}')

    return response

def display_diff(source_str, result_str):
    dmp = dmp_module.diff_match_patch()
    diffs = dmp.diff_main(source_str, result_str)
    dmp.diff_cleanupSemantic(diffs)
    html_diff = dmp.diff_prettyHtml(diffs)
    return html_diff

In [ ]:
chat_promt = '''
Ты профессиональный редактор и корректор, с филологическим образованием.
Ты работаешь в HR-отделе компании, занимающейяся трудоустройством кандидатов по профилю IT и нейронные сети.
Ты занимаешься исправлением и вычиткой текстов резюме и сопроводительных писем к резюме на вакансию.
К тебе приходит текст резюме, твоя задача - исправить опечатки, орфографию и пунктуацию, а так же мелкие смысловые и стилистические неточности, такие как несогласованность времен при перечислении навыков. Не меняй форматирование, общий смысла предложений и технические термины. Технические термины по возможности нужно приводить к виду, общепринятому в профессиональной среде.
Переносы строки и другое форматирование, такое как html-теги и нумерация пунктов, нужно в точности сохранять, кроме случаев, когда ты исправляешь стилистическую или синтакическую ошибку.
В твоем ответе должен быть только исправленный текст, без каких-либо добавлений и пояснений с твоей стороны.
'''

resume_text = '''
<h3>2. Личные качества:</h3>

Я спецеалист в области IT, большой опыт работы в сфере информационных тихнологий. Отличные каммуникативные навыки. Высокий организованность и ответственный. Умение работать в команде. Желание дальше развиваться, участвовать в новых проектах и получаю навыки, необходимые для достижения поставленной цели.

<h3>3. Профессиональные навыки:</h3>

Системное администрирование, программирование, ремонтирую компьютерную технику, консультирование, полное техническое обслуживание и поддержка компаний и частных клиентов, развитие it инфраструктуры, развитие бизнес-процессов -системные интеграции.

CRM системы, майнинг фермы, специфическое оборудование. Информационная безопасность, шифрование.

Базовые знания технологий и протоколов:
<ul>
<li>AD,</li> <li>DNS,</li> <li>DHCP,</li> <li>XREGD,</li> <li>FTP,</li> <li>SMTP.</li>
</ul>

Обслуживание и администрирование: 1с, кассовое оборудование, видеонаблюдение, компьютерная техника, сервера, торговое оборудование, аудио-видео техника.

Умение общаться с неопытными пользователями, умею объяснять пользователям сложные вещи простыми словами.

Компьютерные навыки:

Администрирование MS-DOS, Windows 95, 98, 2000, XP, Windows 7, 8, 10, 11, Linux, Windows Server 2000-2003-2008-2012-2016, 1С.
'''
resume_text = resume_text.strip()

In [ ]:
ans=answer_index(
    chat_promt,
    resume_text,
    verbose = 1
)



===========================================: 
1006 токенов ориентировочная длина вопроса

===========================================: 
1005+557=1562 токенов использовано всего (вопрос+ответ).

===========================================: 
Цена запроса с ответом : $ 0.0026, 0.34 ₽

===========================================: 
Исходное резюме:
<h3>2. Личные качества:</h3>

Я специалист в области IT, имею большой опыт работы в сфере информационных технологий. Обладаю отличными коммуникативными навыками. Высокоорганизованный и ответственный. Умею работать в команде. Имею желание дальше развиваться, участвовать в новых проектах и получать навыки, необходимые для достижения поставленной цели.

<h3>3. Профессиональные навыки:</h3>

- Системное администрирование
- Программирование
- Ремонт компьютерной техники
- Консультирование
- Полное техническое обслуживание и поддержка компаний и частных клиентов
- Развитие IT инфраструктуры
- Развитие бизнес-процессов и системные интеграции
- CRM сист

In [ ]:
display(HTML(display_diff(resume_text, ans)))